In [85]:
import cv2
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from matplotlib import pyplot as plt
import pandas as pd

In [136]:
test_dir = "/Users/davidnull/phd/3D Module/data/force/testing/"

# test_name = "10.3cm_3a_test5"
# test_name = "10.3cm_1a_test2"
# test_name = "10.3cm_2a_test2"

# test_name = "14cm_1a_test1"
# test_name = "14cm_2a_test1"
# test_name = "14cm_3a_test1"

# test_name = "18cm_1a_test1"
# test_name = "18cm_2a_test1"
# test_name = "18cm_3a_test2"

# test_name = "re10cm_1a_test1"
# test_name = "re10cm_2a_test1"
# test_name = "re10cm_3a_test1"

# test_name = "re14cm_1a_test1"
# test_name = "re14cm_2a_test1"
test_name = "re14cm_3a_test1"

# test_name = "re18cm_1a_test1"
# test_name = "re18cm_2a_test1"
# test_name = "re18cm_3a_test1"

csv_ext = ".csv"
mov_ext = ".MOV"

csv_file = test_dir + test_name + csv_ext
movie_file = test_dir + test_name + mov_ext

vid = cv2.VideoCapture(movie_file)

fps = round(vid.get(5))
print('Frames per second : ', fps,'FPS')

frame_count = vid.get(7)
print('Frame count : ', frame_count)

df = pd.read_csv(csv_file)
df = df.loc[df['stage'] == 6]
df = df.reset_index(drop=True)
exp_length = len(df)
df = df.assign(seconds=[0]*exp_length)
df = df.assign(sync=[0]*exp_length)
df = df.assign(force=[0]*exp_length)

df["seconds"] = df["timestamp"] / 1000000
start_time = df["seconds"].iloc[0]
df["seconds"] = df["seconds"] - start_time
print(df)

Frames per second :  30 FPS
Frame count :  4066.0
      stage     id    ping       mem    memExt  timestamp  init  drivers  \
0         6  45699  114258  0.622499  0.005118  836578153   255        0   
1         6  45699  114258  0.622499  0.005118  836602667   255        0   
2         6  45699  114258  0.622499  0.005118  836627432   255        0   
3         6  45699  114258  0.622499  0.005118  836652508   255        0   
4         6  45699  114258  0.622499  0.005118  836682314   255        0   
...     ...    ...     ...       ...       ...        ...   ...      ...   
4377      6  45699   76000  0.624352  0.005118  964602239   255        0   
4378      6  45699   76000  0.624352  0.005118  964627432   255        0   
4379      6  45699   77354  0.624352  0.005118  964658356   255        0   
4380      6  45699   77354  0.624352  0.005118  964679426   255        0   
4381      6  45699   77354  0.624352  0.005118  964703177   255        0   

              p0         p1          

In [134]:
@interact(sync="", force="", frame_id=(0,frame_count,1))
def f(sync, force, frame_id):
    vid.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
    ret, frame = vid.read()
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rotated = cv2.rotate(rgb_frame, cv2.ROTATE_90_CLOCKWISE)
    rotated = cv2.rotate(rotated, cv2.ROTATE_90_CLOCKWISE)
    cropped = rotated[300:800, 500:1000]
    plt.imshow(cropped)
    plt.show()
    if sync == "" or force == "":
        return
    sync = float(sync)
    force = float(force)
    if frame_id < sync:
        return 
    frames_since = frame_id - sync
    time_since = (1/fps) * frames_since
    place_id = (df['seconds']-time_since).abs().argsort()[0]
    df.at[place_id, "force"] = force
    df.at[place_id, "sync"] = sync
    return

interactive(children=(Text(value='', description='sync'), Text(value='', description='force'), FloatSlider(val…

1. sync frame stamp
2. match force data

In [135]:
df.to_csv(test_name + "_wforce" + csv_ext)